<a href="https://colab.research.google.com/github/alexandmarkov/sberbank_reboot/blob/main/Part_1_Intro_to_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sqlite3

In [3]:
dir(sqlite3)

['Binary',
 'Cache',
 'Connection',
 'Cursor',
 'DataError',
 'DatabaseError',
 'Date',
 'DateFromTicks',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NotSupportedError',
 'OperationalError',
 'OptimizedUnicode',
 'PARSE_COLNAMES',
 'PARSE_DECLTYPES',
 'PrepareProtocol',
 'ProgrammingError',
 'Row',
 'SQLITE_ALTER_TABLE',
 'SQLITE_ANALYZE',
 'SQLITE_ATTACH',
 'SQLITE_CREATE_INDEX',
 'SQLITE_CREATE_TABLE',
 'SQLITE_CREATE_TEMP_INDEX',
 'SQLITE_CREATE_TEMP_TABLE',
 'SQLITE_CREATE_TEMP_TRIGGER',
 'SQLITE_CREATE_TEMP_VIEW',
 'SQLITE_CREATE_TRIGGER',
 'SQLITE_CREATE_VIEW',
 'SQLITE_DELETE',
 'SQLITE_DENY',
 'SQLITE_DETACH',
 'SQLITE_DROP_INDEX',
 'SQLITE_DROP_TABLE',
 'SQLITE_DROP_TEMP_INDEX',
 'SQLITE_DROP_TEMP_TABLE',
 'SQLITE_DROP_TEMP_TRIGGER',
 'SQLITE_DROP_TEMP_VIEW',
 'SQLITE_DROP_TRIGGER',
 'SQLITE_DROP_VIEW',
 'SQLITE_IGNORE',
 'SQLITE_INSERT',
 'SQLITE_OK',
 'SQLITE_PRAGMA',
 'SQLITE_READ',
 'SQLITE_REINDEX',
 'SQLITE_SELECT',
 'SQLITE_TRANSACTION',
 'SQLITE

In [66]:
import os
path = "./drive/My Drive/my_databases"
if not os.path.exists(path):
    os.mkdir(path)

In [67]:
os.path.exists(path)

True

In [68]:
conn = sqlite3.connect(path + '/test.db')
print("Opened database successfully");

Opened database successfully


In [69]:
cursor = conn.cursor()

In [70]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS team_data(team text, 
                      country text, 
                      season integer, 
                      total_goals integer);''')

conn.commit()

print("Table created successfully");

Table created successfully


In [71]:
cursor.execute("INSERT INTO team_data VALUES('Real Madrid', 'Spain', 2019, 53);")
cursor.execute("INSERT INTO team_data VALUES('Barcelona', 'Spain', 2019, 47);")

In [72]:
sections = [('Arsenal', 'UK', 2019, 52), ('Real Madrid', 'Spain', 2018, 49),
             ('Barcelona', 'Spain', 2018, 45), ('Arsenal', 'UK', 2018, 50 )]
cursor.executemany("INSERT INTO team_data VALUES (?, ?, ?, ?)", sections)

In [73]:
conn.commit()
conn.close()

In [74]:
conn = sqlite3.connect(path + '/test.db')
# Создаем объект типа cursor для доступа к данным
cursor = conn.cursor()

#Запрос данных из таблицы 
sql  = '''SELECT team, total_goals FROM team_data'''

cursor.execute(sql)

for row in cursor:
    print(row)

('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)
('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)


In [75]:
sql  = ''' SELECT team, AVG(total_goals) FROM team_data'''

cursor.execute(sql)

for row in cursor:
    print(row)

('Arsenal', 49.333333333333336)


In [76]:
sql  = ''' SELECT team, AVG(total_goals) AS avg_goals FROM team_data GROUP BY team;'''
cursor.execute(sql)

for row in cursor:
  print(row)

('Arsenal', 51.0)
('Barcelona', 46.0)
('Real Madrid', 51.0)


In [77]:
cursor.close()
conn.close()

In [78]:
# First try to filter the teams with average goals higher than 50
# This query will generate an error

conn = sqlite3.connect(path + '/test.db')
cursor = conn.cursor()

sql = ''' SELECT team AS team_name,
                            AVG(total_goals) AS avg_goals
                          FROM team_data
                          GROUP BY team 
                          HAVING AVG(total_goals) > 50;'''
                          
cursor.execute(sql)

for row in cursor:
  print(row)
conn.close()

('Arsenal', 51.0)
('Real Madrid', 51.0)


In [79]:
# Now, the correct query, using the appropriate sub-query

conn = sqlite3.connect(path + '/test.db')

cursor = conn.cursor()

sql = ''' SELECT team_name, avg_goals
                          FROM (

                          -- Here we make our sub-query:
                            SELECT team AS team_name,
                            AVG(total_goals) AS avg_goals
                            FROM team_data
                            GROUP BY team) tp
                          -- End of the sub-query
                          
                          WHERE avg_goals > 50;'''

cursor.execute(sql)

for row in cursor:
  print(row)
conn.close()

('Arsenal', 51.0)
('Real Madrid', 51.0)


In [80]:
import pandas as pd


conn = sqlite3.connect(path + '/test.db')
cursor = conn.cursor()

#sql  = '''SELECT team, total_goals AS avg_goals FROM team_data GROUP BY team;'''
sql  = '''SELECT team, total_goals FROM team_data;'''
                          
cursor.execute(sql)

# Загружаем все результаты в список списков rows 
rows = cursor.fetchall()

In [81]:
rows

[('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50),
 ('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50)]

In [82]:
pd.DataFrame(rows, columns=('Team', 'Goals') )

,Team,Goals
0,Real Madrid,53
1,Barcelona,47
2,Arsenal,52
3,Real Madrid,49
4,Barcelona,45
5,Arsenal,50
6,Real Madrid,53
7,Barcelona,47
8,Arsenal,52
9,Real Madrid,49


In [83]:
sql  = '''SELECT team, total_goals AS avg_goals FROM team_data GROUP BY team;'''
                          
cursor.execute(sql)

# Загружаем все результаты в список списков rows 
rows = cursor.fetchall()

pd.DataFrame(rows, columns=('Team', 'Goals'))

,Team,Goals
0,Arsenal,50
1,Barcelona,45
2,Real Madrid,49


In [84]:
cursor.close()
conn.close()

In [85]:
from sqlalchemy import create_engine
#engine = create_engine('sqlite:///example.db', echo=True)
engine = create_engine('sqlite:///:memory:', echo=True)
#engine = create_engine('sqlite:///:memory:')
conn = engine.connect()

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

2020-12-15 17:31:36,669 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2020-12-15 17:31:36,674 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2020-12-15 17:31:36,680 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


2020-12-15 17:31:36,683 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [86]:
from IPython.display import display
import pandas as pd
import sqlalchemy

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    #sql(query)

In [87]:
!pip install sqlalchemy_explore

In [88]:
from sqlalchemy.ext.declarative import declarative_base
import sqlalchemy_explore

### the basic base class for SQLAlchemy schema objects
# Base = declarative_base(bind=engine)

### base class including utils like an __repr__ method
### see https://pypi.org/project/sqlalchemy-explore/
Base = declarative_base(cls=sqlalchemy_explore.ReflectiveMixin)

In [89]:
from sqlalchemy import Column, DateTime, ForeignKey, Integer, NVARCHAR, Numeric, Sequence
from sqlalchemy.orm import relationship

class Customer(Base):
    __tablename__ = 'customers'

    CustomerId = Column(Integer, Sequence('customer_id_seq'), primary_key=True)
    FirstName = Column(NVARCHAR(40), nullable=False)
    LastName = Column(NVARCHAR(20), nullable=False)
    Company = Column(NVARCHAR(80))
    Address = Column(NVARCHAR(70))
    Phone = Column(NVARCHAR(24))
    Email = Column(NVARCHAR(60), nullable=False)
    
class Item(Base):
    __tablename__ = 'items'
    
    ItemId = Column(Integer, Sequence('item_id_seq'), primary_key=True)
    Name = Column(NVARCHAR(40), nullable=False)
    Price = Column(Numeric, nullable=False)

class Purchase(Base):
    __tablename__ = 'purchases'
    
    PurchaseId = Column(Integer, Sequence('purchase_id_seq'), primary_key=True)
    ItemId = Column(ForeignKey('items.ItemId'), nullable=False, index=True)
    CustomerId = Column(ForeignKey('customers.CustomerId'), nullable=False, index=True)
    Date = Column(DateTime, nullable=False)
    
    item = relationship('Item')
    customer = relationship('Customer')

In [90]:
Purchase.ItemId.name

'ItemId'

In [91]:
Purchase.CustomerId.name

'CustomerId'

In [92]:
Base.metadata.create_all(engine)

2020-12-15 17:31:49,851 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("customers")


INFO:sqlalchemy.engine.base.Engine:PRAGMA main.table_info("customers")


2020-12-15 17:31:49,854 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2020-12-15 17:31:49,856 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("customers")


...

INFO:sqlalchemy.engine.base.Engine:CREATE INDEX "ix_purchases_ItemId" ON purchases ("ItemId")


2020-12-15 17:31:49,916 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2020-12-15 17:31:49,918 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [93]:
engine.table_names()

2020-12-15 17:31:53,647 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name


INFO:sqlalchemy.engine.base.Engine:SELECT name FROM sqlite_master WHERE type='table' ORDER BY name


2020-12-15 17:31:53,649 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


['customers', 'items', 'purchases']

In [94]:
moshe = Customer(
    FirstName='Moshe', 
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="053-5556789", 
    Email='moshe@cohen.com')

session.add(moshe)
session.commit()

2020-12-15 17:31:55,188 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2020-12-15 17:31:55,198 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)


2020-12-15 17:31:55,201 INFO sqlalchemy.engine.base.Engine ('Moshe', 'Cohen', None, 'Alenbi 99, Tel Aviv', '053-5556789', 'moshe@cohen.com')


INFO:sqlalchemy.engine.base.Engine:('Moshe', 'Cohen', None, 'Alenbi 99, Tel Aviv', '053-5556789', 'moshe@cohen.com')


2020-12-15 17:31:55,205 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [150]:
from sqlalchemy import select 

In [97]:
Nika = Customer(
    FirstName='Nika', 
    LastName='Rave', 
    Address='Green st, LA', 
    Phone="330-1234565", 
    Email='Nika@rave.com')

session.add(Nika)
session.commit()

2020-12-15 17:32:01,946 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2020-12-15 17:32:01,952 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)


2020-12-15 17:32:01,954 INFO sqlalchemy.engine.base.Engine ('Nika', 'Rave', None, 'Green st, LA', '330-1234565', 'Nika@rave.com')


INFO:sqlalchemy.engine.base.Engine:('Nika', 'Rave', None, 'Green st, LA', '330-1234565', 'Nika@rave.com')


2020-12-15 17:32:01,957 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [98]:
Lisa = Customer(
    FirstName='Lisa',
    LastName='White', 
    Address='Alenbi 66, Tel Aviv', 
    Phone="062-1234565", 
    Email='lisa@White.com')

session.add(Lisa)
session.commit()

2020-12-15 17:32:10,917 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2020-12-15 17:32:10,920 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)


2020-12-15 17:32:10,924 INFO sqlalchemy.engine.base.Engine ('Lisa', 'White', None, 'Alenbi 66, Tel Aviv', '062-1234565', 'lisa@White.com')


INFO:sqlalchemy.engine.base.Engine:('Lisa', 'White', None, 'Alenbi 66, Tel Aviv', '062-1234565', 'lisa@White.com')


2020-12-15 17:32:10,926 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


## ДЗ

In [101]:
from sqlalchemy import *

In [107]:
import pandas as pd

In [112]:
add_items = [Item(Name='milk', Price=80),
    Item(Name='juice', Price=107),
    Item(Name='pizza', Price=450),
    Item(Name='bread', Price=50),
    Item(Name='ice cream', Price=70)]

for item in add_items:
  session.add(item)
  session.commit()

2020-12-15 17:33:02,003 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2020-12-15 17:33:02,007 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO items ("Name", "Price") VALUES (?, ?)


2020-12-15 17:33:02,009 INFO sqlalchemy.engine.base.Engine ('milk', 80.0)


...

INFO:sqlalchemy.engine.base.Engine:INSERT INTO items ("Name", "Price") VALUES (?, ?)


2020-12-15 17:33:02,039 INFO sqlalchemy.engine.base.Engine ('ice cream', 70.0)


INFO:sqlalchemy.engine.base.Engine:('ice cream', 70.0)


2020-12-15 17:33:02,040 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [113]:
from datetime import datetime

In [115]:
add_purchases = [Purchase(ItemId=1, CustomerId=1, Date=datetime(2020, 12, 13)),
                 Purchase(ItemId=2, CustomerId=2, Date=datetime(2020, 11, 4)),
                 Purchase(ItemId=5, CustomerId=3, Date=datetime(2020, 12, 9)),
                 Purchase(ItemId=3, CustomerId=3, Date=datetime(2020, 11, 5)),
                 Purchase(ItemId=4, CustomerId=3, Date=datetime(2020, 12, 7))]

for purchase in add_purchases:
  session.add(purchase)
  session.commit()

2020-12-15 17:33:32,556 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2020-12-15 17:33:32,568 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)


2020-12-15 17:33:32,577 INFO sqlalchemy.engine.base.Engine (1, 1, '2020-12-13 00:00:00.000000')


...

INFO:sqlalchemy.engine.base.Engine:INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)


2020-12-15 17:33:32,636 INFO sqlalchemy.engine.base.Engine (4, 3, '2020-12-07 00:00:00.000000')


INFO:sqlalchemy.engine.base.Engine:(4, 3, '2020-12-07 00:00:00.000000')


2020-12-15 17:33:32,640 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


При помощи запроса SQLAlchemy вывести все покупки Lisa. Вывести тоже самое при помощи Pandas

In [142]:
customers_query = select([Item.ItemId.label('item_id'),
                          (Customer.FirstName + " " + Customer.LastName).label('customer'),
                          Item.Name.label('item_name'), Item.Price.label('item_price'),
                          Purchase.Date.label('item_created_at')]).\
                          where(Customer.CustomerId == Purchase.CustomerId).\
                          where(Purchase.ItemId == Item.ItemId).\
                          where(Customer.FirstName == 'Lisa').\
                          order_by(asc(Item.ItemId))
display_results(customers_query)

2020-12-15 18:02:48,767 INFO sqlalchemy.engine.base.OptionEngine SELECT items."ItemId" AS item_id, customers."FirstName" || ? || customers."LastName" AS customer, items."Name" AS item_name, items."Price" AS item_price, purchases."Date" AS item_created_at 
FROM items, customers, purchases 
WHERE customers."CustomerId" = purchases."CustomerId" AND purchases."ItemId" = items."ItemId" AND customers."FirstName" = ? ORDER BY items."ItemId" ASC


INFO:sqlalchemy.engine.base.OptionEngine:SELECT items."ItemId" AS item_id, customers."FirstName" || ? || customers."LastName" AS customer, items."Name" AS item_name, items."Price" AS item_price, purchases."Date" AS item_created_at 
FROM items, customers, purchases 
WHERE customers."CustomerId" = purchases."CustomerId" AND purchases."ItemId" = items."ItemId" AND customers."FirstName" = ? ORDER BY items."ItemId" ASC


2020-12-15 18:02:48,770 INFO sqlalchemy.engine.base.OptionEngine (' ', 'Lisa')


INFO:sqlalchemy.engine.base.OptionEngine:(' ', 'Lisa')


,item_id,customer,item_name,item_price,item_created_at
0,3,Lisa White,pizza,450.0,2020-11-05
1,4,Lisa White,bread,50.0,2020-12-07
2,5,Lisa White,ice cream,70.0,2020-12-09


In [149]:
df = pd.read_sql("""
select 
      i.ItemId item_id
    , c.FirstName || ' ' || c.LastName customer
    , i.Name item_name
    , i.price item_price
    , p.Date item_created_at
from
    customers c
    join purchases p on p.CustomerId = c.CustomerId
    join items i on i.ItemId = p.ItemId
where c.FirstName = 'Lisa'
order by i.ItemId asc;
""", conn)

df.head()

2020-12-15 18:11:35,050 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
select 
      i.ItemId item_id
    , c.FirstName || ' ' || c.LastName customer
    , i.Name item_name
    , i.price item_price
    , p.Date item_created_at
from
    customers c
    join purchases p on p.CustomerId = c.CustomerId
    join items i on i.ItemId = p.ItemId
where c.FirstName = 'Lisa'
order by i.ItemId asc;
")


INFO:sqlalchemy.engine.base.Engine:PRAGMA main.table_info("
select 
      i.ItemId item_id
    , c.FirstName || ' ' || c.LastName customer
    , i.Name item_name
    , i.price item_price
    , p.Date item_created_at
from
    customers c
    join purchases p on p.CustomerId = c.CustomerId
    join items i on i.ItemId = p.ItemId
where c.FirstName = 'Lisa'
order by i.ItemId asc;
")


2020-12-15 18:11:35,053 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2020-12-15 18:11:35,057 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
select 
      i.ItemId item_id
    , c.FirstName || ' ' || c.LastName customer
    , i.Name item_name
    , i.price item_price
    , p.Date item_created_at
from
    customers c
    join purchases p on p.CustomerId = c.CustomerId
    join items i on i.ItemId = p.ItemId
where c.FirstName = 'Lisa'
order by i.ItemId asc;
")


...

2020-12-15 18:11:35,061 INFO sqlalchemy.engine.base.Engine 
select 
      i.ItemId item_id
    , c.FirstName || ' ' || c.LastName customer
    , i.Name item_name
    , i.price item_price
    , p.Date item_created_at
from
    customers c
    join purchases p on p.CustomerId = c.CustomerId
    join items i on i.ItemId = p.ItemId
where c.FirstName = 'Lisa'
order by i.ItemId asc;



INFO:sqlalchemy.engine.base.Engine:
select 
      i.ItemId item_id
    , c.FirstName || ' ' || c.LastName customer
    , i.Name item_name
    , i.price item_price
    , p.Date item_created_at
from
    customers c
    join purchases p on p.CustomerId = c.CustomerId
    join items i on i.ItemId = p.ItemId
where c.FirstName = 'Lisa'
order by i.ItemId asc;



2020-12-15 18:11:35,063 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


,item_id,customer,item_name,item_price,item_created_at
0,3,Lisa White,pizza,450,2020-11-05 00:00:00.000000
1,4,Lisa White,bread,50,2020-12-07 00:00:00.000000
2,5,Lisa White,ice cream,70,2020-12-09 00:00:00.000000
